In [1]:
import pandas as pd
import numpy as np
import os, re
from typing import Union
from typing import Optional, Tuple

In [2]:
df_fd = pd.read_csv("ignore/fd_summary.csv")
df_fd.head()

,metric,avg,std,cv,notes,prob_name,domain,planner,config
0,translate,0.339556,0.023291,0.068593,NaN,prob10:,zenotravel,fd,lmcut
1,scoping,2.395337,0.073150,0.030538,NaN,prob10:,zenotravel,fd,lmcut
2,translate_and_scope,2.734893,0.081340,0.029741,NaN,prob10:,zenotravel,fd,lmcut
3,plan_unscoped_time,37.723080,0.721374,0.019123,NaN,prob10:,zenotravel,fd,lmcut
4,plan_scoped_time,33.154035,0.405908,0.012243,NaN,prob10:,zenotravel,fd,lmcut


In [3]:
import itertools as it

In [11]:
def clean_col_name(s: str) -> str:
    return s.replace("_"," ").title()
def get_scoped_unscoped_names(s: str):
    return s + " (Scoped)", s + " (Unscoped)"

fd_colmap_manual = {
    "plan_unscoped_generated_nodes": "Eval. (Unscoped)",
    "plan_unscoped_node_expansions": "Exp. (Unscoped)",
    "plan_scoped_generated_nodes": "Eval. (Scoped)",
    "plan_scoped_node_expansions": "Exp. (Scoped)",
    "plan_unscoped_time": "Planning (Unscoped)",
    "plan_scoped_time": "Planning (Scoped)",
    "encoding_size": "Encoding Size",
    "total_unscoped_time": "Total (Unscoped)",
    "total_scoped_time": "Total (Scoped)"
}
paired_metrics =  ["Eval.", "Exp.", "Total", "Planning" ]
time_metrics = ["Total", "Planning", "Translate", "Scoping"]
time_metrics_scoped_and_unscoped = list(it.chain(*[get_scoped_unscoped_names(m) for m in time_metrics]))
solo_metrics = ["Translate", "Scoping"]

fd_colmap = {c: clean_col_name(c) for c in df_fd["metric"].unique()}
fd_colmap.update(fd_colmap_manual)
# df_fd = df_fd.rename(columns=fd_colmap)
# df_fd
df_fd["metric"] = df_fd["metric"].replace(fd_colmap)
config2keep = "ms"
df_fd = df_fd[df_fd["config"] == config2keep]
df_fd

,metric,avg,std,cv,notes,prob_name,domain,planner,config
36,Translate,2.732207e-01,0.017759,0.065000,NaN,prob15:,logistics,fd,ms
37,Scoping,6.790909e-01,0.032256,0.047499,NaN,prob15:,logistics,fd,ms
38,Translate And Scope,9.523117e-01,0.049447,0.051923,NaN,prob15:,logistics,fd,ms
39,Planning (Unscoped),7.371776e+01,2.922776,0.039648,NaN,prob15:,logistics,fd,ms
40,Planning (Scoped),1.113938e+01,0.796540,0.071507,NaN,prob15:,logistics,fd,ms
...,...,...,...,...,...,...,...,...,...
247,Eval. (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms
248,Exp. (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms
249,Eval. (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms
250,Exp. (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms


In [25]:
def parse_failure(df: pd.DataFrame, c: str) -> pd.DataFrame:
    df = df.copy()
    c_fail = "Failed"
    df[c_fail] = df[c].astype(str).str.startswith(">")
    df[c_fail] = df[c_fail] | df[c].isna()
    m = df[c_fail]
    df.loc[m, c] = df.loc[m, c].str.slice(1)
    df[c] = df[c].astype(float)
    return df

In [26]:
# def parse_timeouts(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
#     df = df.copy()
#     for c in cols:
#         c_timeout = c + " (Timed Out)"
#         df[c_timeout] = df[c].astype(str).str.startswith(">")
#         m = df[c_timeout]
#         df.loc[m, c] = df.loc[m, c].str.slice(1)
#         df[c] = df[c].astype(float)
#     return df

In [27]:
# df = parse_timeouts(df_fd, time_metrics_scoped_and_unscoped)
df = parse_failure(df_fd, "avg")
df

,metric,avg,std,cv,notes,prob_name,domain,planner,config,Failed
36,Translate,0.273221,0.017759,0.065000,NaN,prob15:,logistics,fd,ms,False
37,Scoping,0.679091,0.032256,0.047499,NaN,prob15:,logistics,fd,ms,False
38,Translate And Scope,0.952312,0.049447,0.051923,NaN,prob15:,logistics,fd,ms,False
39,Planning (Unscoped),73.717760,2.922776,0.039648,NaN,prob15:,logistics,fd,ms,False
40,Planning (Scoped),11.139380,0.796540,0.071507,NaN,prob15:,logistics,fd,ms,False
...,...,...,...,...,...,...,...,...,...,...
247,Eval. (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
248,Exp. (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
249,Eval. (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
250,Exp. (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True


In [29]:
df[df["Failed"]]

,metric,avg,std,cv,notes,prob_name,domain,planner,config,Failed
171,Planning (Unscoped),NaN,NaN,NaN,(100% mem-out),prob07:,satellite,fd,ms,True
173,Total (Unscoped),8.381490e+02,18.28872,NaN,(100% mem-out),prob07:,satellite,fd,ms,True
175,Eval. (Unscoped),3.623950e+08,NaN,NaN,(100% mem-out),prob07:,satellite,fd,ms,True
176,Exp. (Unscoped),2.142340e+07,NaN,NaN,(100% mem-out),prob07:,satellite,fd,ms,True
243,Planning (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
244,Planning (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
245,Total (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
246,Total (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
247,Eval. (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True
248,Exp. (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms,True


In [ ]:
def format_float(a: float, n_dec: Optional[int]) -> str:
    a = round(a, n_dec)
    if n_dec is None:
        a = int(a)
        a = f"{a:,}"
    else:
        a = f"{a:,.1f}"
    return a

def get_table_str(df: pd.DataFrame, metric: str, use_sd: bool = False) -> str:
    n_dec = None
    long_dec_substrings = ["translate","scoping", "plan", "total"]
    for s in long_dec_substrings:
        if s in metric.lower():
            n_dec = 1
    a = df.loc[metric, "avg"]
    if isinstance(a, str):
        try:
            a = float(a)
        except:
            pass
    if isinstance(a, (float,)):
        # a = str(round(a, 2))
        a = format_float(a, n_dec=n_dec)
    sd = df.loc[metric, "std"]
    if isinstance(sd, str):
        try:
            sd = float(sd)
        except:
            pass
    if isinstance(sd, float):
        sd = format_float(sd, n_dec=n_dec)
    if use_sd:
        return f"{a} \pm {sd}"
    else:
        return str(a)
    


# def remove_gt(s: Union[str, float]) -> Tuple[Union[str, float], bool]:
#     """Returns float, and whether it started with >"""
#     if isinstance(s, float):
#         return s, False
#     elif isinstance(s, str):
#         if len(s) > 0 and s[0] == ">":
#             return float(s[1:]), True
#         else:
#             return float(s), False

def get_better_col(df: pd.DataFrame, metric: str) -> Optional[str]:
    metric_scoped, metric_unscoped = get_scoped_unscoped_names(metric)
    v_scoped, v_unscoped = df.loc[metric_scoped, "avg"], df.loc[metric_unscoped, "avg"]
    (v_scoped, gt_scoped) = remove_gt(v_scoped)
    (v_unscoped, gt_unscoped) = remove_gt(v_unscoped)
    # Handle NaN
    if np.isnan(v_scoped) and np.isnan(v_unscoped):
        return None
    if np.isnan(v_scoped) and not np.isnan(v_unscoped):
        return metric_unscoped
    if not np.isnan(v_scoped) and np.isnan(v_unscoped):
        return metric_scoped
    
    # Handle gt
    if gt_scoped and not gt_unscoped:
        return metric_unscoped
    if not gt_scoped and gt_unscoped:
        return metric_scoped

    if v_scoped > v_unscoped:
        return metric_unscoped
    if v_scoped < v_unscoped:
        return metric_scoped
    if v_scoped == v_unscoped:
        return None

In [ ]:

dfs = []
for (domain, prob_name, planner, config), df in df_fd.groupby(["domain", "prob_name", "planner", "config"]):
    df = df.copy()
    df["metric"] = df["metric"].replace(fd_colmap)
    df = df.set_index("metric")
    # display(df)
    prob_num = re.search("\d+", prob_name).group()
    dom_prob = domain + " " + prob_num
    # df_new = pd.DataFrame({"Problem":dom_prob})
    # df_new.loc[]

        
    d = {
        "Problem": [dom_prob],
        # "Config": [config]
    }
    # col2format = {}
    for metric in paired_metrics:
        metric_scoped, metric_unscoped = get_scoped_unscoped_names(metric)
        use_sd = metric in time_metrics
        d[metric_scoped] = [get_table_str(df, metric_scoped, use_sd=use_sd)]
        d[metric_unscoped] = [get_table_str(df, metric_unscoped, use_sd=use_sd)]
        better_column = get_better_col(df, metric)
        if better_column is not None:
            d[better_column] = ["\\mathbf{" + d[better_column][0] + "}"]

    for metric in solo_metrics:
        use_sd = metric in time_metrics
        d[metric] = [get_table_str(df, metric,use_sd=use_sd)]

    metrics_all = solo_metrics
    for c in paired_metrics:
        metrics_all.extend(list(get_scoped_unscoped_names(c)))
        
    for c in metrics_all:
        d[c][0] = "$" + d[c][0] + "$"

    # [dom_prob, "Generated Nodes (Unscoped)", "Generated Nodes (Scoped)"]
    df_new = pd.DataFrame(data=d)
    # display(df_new)
    dfs.append(df_new)

df_out = pd.concat(dfs)
display(df_out)

In [ ]:
pd.read_csv("domain_sizes.csv")

In [ ]:
df_size = pd.read_csv("domain_sizes.csv")
df_out = df_out.set_index("Problem")
# df_size["domprob"] = df_size["domain"].str + df_size["problem"].str.split("prob")[1]
df_size
for ind, r in df_size.iterrows():
    df_size.loc[ind, "Problem"] = df_size.loc[ind, "domain"] + " " + df_size.loc[ind, "problem"].split("prob")[1]
df_size = df_size.drop(columns=["domain", "problem"])
df_size
dfs_size = []
for problem, df in df_size.groupby("Problem"):
    df = df.set_index(["Problem", "scoped"])
    d = {
        # "Problem": problem,
        # "State Space (Scoped)": df.loc[(problem, "scoped"), "state_size"] ,
        # "State Space (Unscoped)": df.loc[(problem, "unscoped"), "state_size"],
        "Operators (Scoped)": df.loc[(problem, "scoped"), "operators"],
        "Operators (Unscoped)": df.loc[(problem, "unscoped"), "operators"]
    }

    # paired_metrics =  ["State Space", "Operators"]
    paired_metrics =  ["Operators"]

    for metric in paired_metrics:
        # Convert to scientific notation for state space?
        metric_scoped, metric_unscoped = get_scoped_unscoped_names(metric)
        if d[metric_scoped] > d[metric_unscoped]:
            d[metric_unscoped] = "\\mathbf{" + str(d[metric_unscoped]) + "}"
            d[metric_scoped] = str(d[metric_scoped])
        elif d[metric_scoped] < d[metric_unscoped]:
            d[metric_scoped] = "\\mathbf{" + str(d[metric_scoped]) + "}"
            d[metric_unscoped] = str(d[metric_unscoped])
        else:
            d[metric_scoped] = str(d[metric_unscoped])
            d[metric_scoped] = str(d[metric_scoped])


    for k,v in d.items():
        d[k] = "$" + v + "$"
        df_out.loc[problem, k] = d[k]
    # df = pd.DataFrame({k: [v] for k, v in d.items()})
    # dfs_size.append(df)
# df_size = pd.concat(dfs_size)
# display(df_size)
cols = ["Problem", "Operators (Unscoped)", "Operators (Scoped)", "Exp. (Unscoped)", "Exp. (Scoped)", "Eval. (Unscoped)", "Eval. (Scoped)", "Translate", "Scoping", "Planning (Unscoped)", "Planning (Scoped)", "Total (Unscoped)", "Total (Scoped)"]
df_out = df_out.reset_index(drop=False)[cols]
df_out

In [ ]:
def df2tex(df: pd.DataFrame, label: str, caption: str) -> str:
    
    # Header
    # rows = ["\\begin{table*}[t]","\\resizebox{\\textwidth}{!}{%","\\begin{tabular}{" + "l"*len(df.columns) + "}"]
    # rows.append("\\toprule")
    # rows.append(" & ".join(df.columns) + "\\\\ \\midrule")
    rows = [r"""\begin{table*}[t]
\resizebox{\textwidth}{!}{%
\begin{tabular}{lrrrrrrrrrrrrr}
\toprule
\textbf{Problem} & \multicolumn{2}{c}{\textbf{Operators}} & \multicolumn{2}{c}{\textbf{Expansions}} & \multicolumn{2}{c}{\textbf{Evaluations}} & \textbf{Translate} & \textbf{Scoping} & \multicolumn{2}{c}{\textbf{Planning Time (s)}} & \multicolumn{2}{c}{\textbf{Total Time (s)}}\\
  & Unscoped & Scoped & Unscoped & Scoped & Unscoped & Scoped &  &  & Unscoped & Scoped & Unscoped & Scoped\\
\midrule"""]

    # Rows
    for ind, r in df.iterrows():
        rows.append(" & ".join(list(r.values)) + "\\\\")
    rows.append("\\bottomrule")
    rows.append("\\end{tabular}")
    rows.append("}")
    rows += ["\\caption{" + caption + "}",
    "\\label{" + label + "}", 
    "\\end{table*}"]
    return "\n".join(rows)

In [ ]:
s_out = df2tex(df_out, label="table:fd_experiments", caption="Results for our Fast Downward experiments.")
with open("tmp.tex", "w") as f:
    f.write(s_out)

Only include lmcut in the main paper, remove ms. That can go in the appendix.
Remove Config column
Add some sort of size column(s)
Rename "Generated Nodes (...)" -> "Eval. (...)"
Rename "Node Expansions (...)" -> "Exp. (...)"
Remove decimal point and ± std from all Eval./Exp. columns
If it's still too wide, use a 2-line header
Column order:
cols = ["Problem", "Operators (Scoped)", "Operators (Unscoped)", "Exp. (Scoped)", "Exp. (Unscoped)", "Eval. (Scoped)", "Eval. (Unscoped)", "Translate", "Scoping", "Planning (Scoped)", "Planning (Unscoped)", "Total (Scoped)", "Total (Unscoped)"]